In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''
import cdsapi
import os
import sys
import dask
import numpy as np
import datetime
import calendar


#from esio import download_c3s  # placed below instead

'''
Download models with sea ice forecasts within the c3s forecast data archive.

https://software.ecmwf.int/wiki/display/c3s/Models

'''

'\nDownload models with sea ice forecasts within the c3s forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/c3s/Models\n\n'

In [3]:
# do not run this part when from Jupyter notebook form

# Check user defined configuraiton file
if len(sys.argv) < 2:
    raise ValueError('Requires either one arguments [recent] \n or two [start_year, end_yaer] [start_month, end_month] (inclusive) ')

# Get name of configuration file/module
timeperiod = sys.argv[1]
if timeperiod=='recent':
    cd = datetime.datetime.now()
    years = [cd.year]
    months = [cd.month]
else:
    year_range_in = list(map(int, sys.argv[1].strip('[]').split(',')))
    month_range_in = list(map(int, sys.argv[2].strip('[]').split(',')))
    if (len(year_range_in)!=2) | (len(month_range_in)!=2):
        raise ValueError('Year range and month range must be two values (inclusive)')
    years = np.arange(year_range_in[0], year_range_in[1]+1, 1)
    months = np.arange(month_range_in[0], month_range_in[1]+1, 1)
    assert np.all(months>=0), 'months must be >=0'
    assert np.all(months<=12), 'months must be <=12'

ValueError: invalid literal for int() with base 10: '-f'

In [24]:
    
# # Testing
years = 1993
#years=np.arange(1993,1995,1)
months = np.arange(1,7,1)
months = np.arange(7,13,1)


monthstr = np.char.mod('%02d', months).tolist()

print(years)
print(months)
monthstr

1993
[ 7  8  9 10 11 12]


['07', '08', '09', '10', '11', '12']

In [25]:
@dask.delayed
def download_month(datatype,config_dict,target):
    # Start server
    cserver = cdsapi.Client()
    print("Requesting data with datatype ",datatype)
    print("configuration ",config_dict)
    print("and target ",target)
    cserver.retrieve(datatype,config_dict,target)
    return 1

def download_data_by_month(dataclass=None, datatype=None, main_dir=None,
                           mod_dicts=None, cy=None, cm=None,
                           run_type='forecast'):
    X = 1

    if dataclass=='s2s':
        day_lag = 22
    elif dataclass=='c3':
        day_lag = 16   
    else:
        raise ValueError('dataclass not found.')

    DS = datetime.datetime(cy,cm,1)
    DE = datetime.datetime(cy,cm,calendar.monthrange(cy,cm)[1])

    cd = datetime.datetime.now()
    S2S = cd - datetime.timedelta(days=day_lag)

    # Check if current month, insure dates are not within last 16/21 days (embargo)
    DE = np.min([DE, S2S])

    # Check if most recent init date is before current month start
    if DS>DE:
        print('No data available yet for ', str(cy),'-',str(cm))
        print('Re-downloading previous month...')
        cm = cm -1
        if cm==0:
            cm =  12
            cy = cy - 1
        print(cm)
        download_data_by_month(dataclass=dataclass, datatype=datatype, main_dir=main_dir,
                               mod_dicts=mod_dicts, cy=cy, cm=cm)
        return 0 # Just return an int for dask. Don't continue here.

    for cmod in mod_dicts.keys():
        print(cmod)
        cdict = mod_dicts[cmod]
        # Update cdict
        cdict['year'] = str(cy)
        cdict['month'] = format(cm, '02')
        target = os.path.join(main_dir, cmod, run_type, 'native',
                              cmod+'_'+str(cy)+'_'+format(cm, '02')+'.grib')
        X = X + download_month(datatype,cdict,target)

    # Call compute to download all models concurently from ecmwf
    X.compute()


In [26]:
main_dir = '/home/disk/sipn/nicway/data/model'

In [27]:
# Templet dicts for each model
### ukmetoffice is upgrading to system 14 at the moment only some months are done for retrospective
# so get system 13 and 14 both
# reforecast ukmetoffice only made on 01, 09, 17, 25

# Init it
mod_dicts = {}

mod_dicts['ukmetofficesipn']  =    {'format':'grib',
     'originating_centre':'ukmo',
     'system':'13',
     'variable':'sea_ice_cover',
     'year':'placeholder',
     'month':'placeholder',
     'day':['01','09','17','25'],
     'leadtime_hour':['24','48','72','96','120','144','168','192','216','240','264','288','312','336','360','384','408','432','456','480','504','528','552','576','600','624','648','672','696','720','744','768','792','816','840','864','888','912','936','960','984','1008','1032','1056','1080','1104','1128','1152','1176','1200','1224','1248','1272','1296','1320','1344','1368','1392','1416','1440','1464','1488','1512','1536','1560','1584','1608','1632','1656','1680','1704','1728','1752','1776','1800','1824','1848','1872','1896','1920','1944','1968','1992','2016','2040','2064','2088','2112','2136','2160','2184','2208','2232','2256','2280','2304','2328','2352','2376','2400','2424','2448','2472','2496','2520','2544','2568','2592','2616','2640','2664','2688','2712','2736','2760','2784','2808','2832','2856','2880','2904','2928','2952','2976','3000','3024','3048','3072','3096','3120','3144','3168','3192','3216','3240','3264','3288','3312','3336','3360','3384','3408','3432','3456','3480','3504','3528','3552','3576','3600','3624','3648','3672','3696','3720','3744','3768','3792','3816','3840','3864','3888','3912','3936','3960','3984','4008','4032','4056','4080','4104','4128','4152','4176','4200','4224','4248','4272','4296','4320','4344','4368','4392','4416','4440','4464','4488','4512','4536','4560','4584','4608','4632','4656','4680','4704','4728','4752','4776','4800','4824','4848','4872','4896','4920','4944','4968','4992','5016','5040','5064','5088','5112','5136','5160'],
                                   }

#mod_dicts.keys()

In [28]:
monthstr

['07', '08', '09', '10', '11', '12']

In [29]:
# use this to get multiple months at once, useful since it takes ages to queue
datatype='seasonal-original-single-levels'
cy=str(years)
run_type='reforecast'
X=1
print(cy)
for cmod in mod_dicts.keys():
    print(cmod)
    cdict = mod_dicts[cmod]
    cdict['year'] = str(cy)
    cdict['month'] = monthstr

    target = os.path.join(main_dir, cmod, run_type, 'tmp',
                  cmod+'_'+str(cy)+'_Jan_Jun.grib')

    X = download_month(datatype,cdict,target)

    X.compute()


2019-07-20 17:50:17,611 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels


1993
ukmetofficesipn
Requesting data with datatype  seasonal-original-single-levels
configuration  {'format': 'grib', 'originating_centre': 'ukmo', 'system': '13', 'variable': 'sea_ice_cover', 'year': '1993', 'month': ['07', '08', '09', '10', '11', '12'], 'day': ['01', '09', '17', '25'], 'leadtime_hour': ['24', '48', '72', '96', '120', '144', '168', '192', '216', '240', '264', '288', '312', '336', '360', '384', '408', '432', '456', '480', '504', '528', '552', '576', '600', '624', '648', '672', '696', '720', '744', '768', '792', '816', '840', '864', '888', '912', '936', '960', '984', '1008', '1032', '1056', '1080', '1104', '1128', '1152', '1176', '1200', '1224', '1248', '1272', '1296', '1320', '1344', '1368', '1392', '1416', '1440', '1464', '1488', '1512', '1536', '1560', '1584', '1608', '1632', '1656', '1680', '1704', '1728', '1752', '1776', '1800', '1824', '1848', '1872', '1896', '1920', '1944', '1968', '1992', '2016', '2040', '2064', '2088', '2112', '2136', '2160', '2184', '2208', '2

2019-07-20 17:50:18,582 INFO Request is queued
2019-07-20 17:51:59,620 INFO Request is failed
2019-07-20 17:51:59,621 ERROR Message: Cancelled
2019-07-20 17:51:59,622 ERROR Reason:  Cancelled
2019-07-20 17:52:13,462 INFO Request is running
2019-07-20 18:08:42,937 INFO Request is completed
2019-07-20 18:08:42,938 INFO Downloading http://136.156.133.41/cache-compute-0013/cache/data2/adaptor.mars.external-1563670306.9044256-19035-13-4533c83f-e214-4f46-81c4-f2d59b4cbb2e.grib to /home/disk/sipn/nicway/data/model/ukmetofficesipn/reforecast/tmp/ukmetofficesipn_1993_Jan_Jun.grib (3.4G)
2019-07-20 18:13:15,174 INFO Download rate 12.8M/s


In [ ]:
# this would get them one at a time, taking forever
## Download by month, wait for each month to finish
#for cy in years:
#    print(str(cy))
#    for cm in months:
#        print(cm)
#        download_data_by_month(dataclass='c3', datatype='seasonal-original-single-levels', main_dir=main_dir, 
#                                      mod_dicts=mod_dicts, cy=cy, cm=cm, run_type='reforecast')

In [ ]:


mod_dicts['meteofrsipn']  =    {'format':'grib',
     'originating_centre':'meteo_france',
     'system':'6',
     'variable':'sea_ice_cover',
     'year':'placeholder',
     'month':'placeholder',
     'day':'01',
     'leadtime_hour':['24','48','72',
            '96','120','144',
            '168','192','216',
            '240','264','288',
            '312','336','360',
            '384','408','432',
            '456','480','504',
            '528','552','576',
            '600','624','648',
            '672','696','720',
            '744','768','792',
            '816','840','864',
            '888','912','936',
            '960','984','1008',
            '1032','1056','1080',
            '1104','1128','1152',
            '1176','1200','1224',
            '1248','1272','1296',
            '1320','1344','1368',
            '1392','1416','1440',
            '1464','1488','1512',
            '1536','1560','1584',
            '1608','1632','1656',
            '1680','1704','1728',
            '1752','1776','1800',
            '1824','1848','1872',
            '1896','1920','1944',
            '1968','1992','2016',
            '2040','2064','2088',
            '2112','2136','2160',
            '2184','2208','2232',
            '2256','2280','2304',
            '2328','2352','2376',
            '2400','2424','2448',
            '2472','2496','2520',
            '2544','2568','2592',
            '2616','2640','2664',
            '2688','2712','2736',
            '2760','2784','2808',
            '2832','2856','2880',
            '2904','2928','2952',
            '2976','3000','3024',
            '3048','3072','3096',
            '3120','3144','3168',
            '3192','3216','3240',
            '3264','3288','3312',
            '3336','3360','3384',
            '3408','3432','3456',
            '3480','3504','3528',
            '3552','3576','3600',
            '3624','3648','3672',
            '3696','3720','3744',
            '3768','3792','3816',
            '3840','3864','3888',
            '3912','3936','3960',
            '3984','4008','4032',
            '4056','4080','4104',
            '4128','4152','4176',
            '4200','4224','4248',
            '4272','4296','4320',
            '4344','4368','4392',
            '4416','4440','4464',
            '4488','4512','4536',
            '4560','4584','4608',
            '4632','4656','4680',
            '4704','4728','4752',
            '4776','4800','4824',
            '4848','4872','4896',
            '4920','4944','4968',
            '4992','5016','5040',
            '5064'
        ]

                                   }



In [ ]:
mod_dicts['ukmetofficesipn']  =    {'format':'grib',
     'originating_centre':'ukmo',
     'system':['13'],
     'variable':'sea_ice_cover',
     'year':'placeholder',
     'month':'placeholder',
     'day':['01','09','17','25'],
     'leadtime_hour':['24','48','72','96','120','144','168','192','216','240','264','288','312','336','360','384','408','432','456','480','504','528','552','576','600','624','648','672','696','720','744','768','792','816','840','864','888','912','936','960','984','1008','1032','1056','1080','1104','1128','1152','1176','1200','1224','1248','1272','1296','1320','1344','1368','1392','1416','1440','1464','1488','1512','1536','1560','1584','1608','1632','1656','1680','1704','1728','1752','1776','1800','1824','1848','1872','1896','1920','1944','1968','1992','2016','2040','2064','2088','2112','2136','2160','2184','2208','2232','2256','2280','2304','2328','2352','2376','2400','2424','2448','2472','2496','2520','2544','2568','2592','2616','2640','2664','2688','2712','2736','2760','2784','2808','2832','2856','2880','2904','2928','2952','2976','3000','3024','3048','3072','3096','3120','3144','3168','3192','3216','3240','3264','3288','3312','3336','3360','3384','3408','3432','3456','3480','3504','3528','3552','3576','3600','3624','3648','3672','3696','3720','3744','3768','3792','3816','3840','3864','3888','3912','3936','3960','3984','4008','4032','4056','4080','4104','4128','4152','4176','4200','4224','4248','4272','4296','4320','4344','4368','4392','4416','4440','4464','4488','4512','4536','4560','4584','4608','4632','4656','4680','4704','4728','4752','4776','4800','4824','4848','4872','4896','4920','4944','4968','4992','5016','5040','5064','5088','5112','5136','5160'],
                                   }

In [ ]:
mod_dicts['ecmwfsipn']  =    {'format':'grib',
     'originating_centre':'ecmwf',
     'system':'5',
     'variable':'sea_ice_cover',
     'year':'placeholder',
     'month':'placeholder',
     'day':'01',
     'leadtime_hour':['0','6','12','18','24','30','36','42','48','54','60','66','72','78','84','90','96','102','108','114','120','126','132','138','144','150','156','162','168','174','180','186','192','198','204','210','216','222','228','234','240','246','252','258','264','270','276','282','288','294','300','306','312','318','324','330','336','342','348','354','360','366','372','378','384','390','396','402','408','414','420','426','432','438','444','450','456','462','468','474','480','486','492','498','504','510','516','522','528','534','540','546','552','558','564','570','576','582','588','594','600','606','612','618','624','630','636','642','648','654','660','666','672','678','684','690','696','702','708','714','720','726','732','738','744','750','756','762','768','774','780','786','792','798','804','810','816','822','828','834','840','846','852','858','864','870','876','882','888','894','900','906','912','918','924','930','936','942','948','954','960','966','972','978','984','990','996','1002','1008','1014','1020','1026','1032','1038','1044','1050','1056','1062','1068','1074','1080','1086','1092','1098','1104','1110','1116','1122','1128','1134','1140','1146','1152','1158','1164','1170','1176','1182','1188','1194','1200','1206','1212','1218','1224','1230','1236','1242','1248','1254','1260','1266','1272','1278','1284','1290','1296','1302','1308','1314','1320','1326','1332','1338','1344','1350','1356','1362','1368','1374','1380','1386','1392','1398','1404','1410','1416','1422','1428','1434','1440','1446','1452','1458','1464','1470','1476','1482','1488','1494','1500','1506','1512','1518','1524','1530','1536','1542','1548','1554','1560','1566','1572','1578','1584','1590','1596','1602','1608','1614','1620','1626','1632','1638','1644','1650','1656','1662','1668','1674','1680','1686','1692','1698','1704','1710','1716','1722','1728','1734','1740','1746','1752','1758','1764','1770','1776','1782','1788','1794','1800','1806','1812','1818','1824','1830','1836','1842','1848','1854','1860','1866','1872','1878','1884','1890','1896','1902','1908','1914','1920','1926','1932','1938','1944','1950','1956','1962','1968','1974','1980','1986','1992','1998','2004','2010','2016','2022','2028','2034','2040','2046','2052','2058','2064','2070','2076','2082','2088','2094','2100','2106','2112','2118','2124','2130','2136','2142','2148','2154','2160','2166','2172','2178','2184','2190','2196','2202','2208','2214','2220','2226','2232','2238','2244','2250','2256','2262','2268','2274','2280','2286','2292','2298','2304','2310','2316','2322','2328','2334','2340','2346','2352','2358','2364','2370','2376','2382','2388','2394','2400','2406','2412','2418','2424','2430','2436','2442','2448','2454','2460','2466','2472','2478','2484','2490','2496','2502','2508','2514','2520','2526','2532','2538','2544','2550','2556','2562','2568','2574','2580','2586','2592','2598','2604','2610','2616','2622','2628','2634','2640','2646','2652','2658','2664','2670','2676','2682','2688','2694','2700','2706','2712','2718','2724','2730','2736','2742','2748','2754','2760','2766','2772','2778','2784','2790','2796','2802','2808','2814','2820','2826','2832','2838','2844','2850','2856','2862','2868','2874','2880','2886','2892','2898','2904','2910','2916','2922','2928','2934','2940','2946','2952','2958','2964','2970','2976','2982','2988','2994','3000','3006','3012','3018','3024','3030','3036','3042','3048','3054','3060','3066','3072','3078','3084','3090','3096','3102','3108','3114','3120','3126','3132','3138','3144','3150','3156','3162','3168','3174','3180','3186','3192','3198','3204','3210','3216','3222','3228','3234','3240','3246','3252','3258','3264','3270','3276','3282','3288','3294','3300','3306','3312','3318','3324','3330','3336','3342','3348','3354','3360','3366','3372','3378','3384','3390','3396','3402','3408','3414','3420','3426','3432','3438','3444','3450','3456','3462','3468','3474','3480','3486','3492','3498','3504','3510','3516','3522','3528','3534','3540','3546','3552','3558','3564','3570','3576','3582','3588','3594','3600','3606','3612','3618','3624','3630','3636','3642','3648','3654','3660','3666','3672','3678','3684','3690','3696','3702','3708','3714','3720','3726','3732','3738','3744','3750','3756','3762','3768','3774','3780','3786','3792','3798','3804','3810','3816','3822','3828','3834','3840','3846','3852','3858','3864','3870','3876','3882','3888','3894','3900','3906','3912','3918','3924','3930','3936','3942','3948','3954','3960','3966','3972','3978','3984','3990','3996','4002','4008','4014','4020','4026','4032','4038','4044','4050','4056','4062','4068','4074','4080','4086','4092','4098','4104','4110','4116','4122','4128','4134','4140','4146','4152','4158','4164','4170','4176','4182','4188','4194','4200','4206','4212','4218','4224','4230','4236','4242','4248','4254','4260','4266','4272','4278','4284','4290','4296','4302','4308','4314','4320','4326','4332','4338','4344','4350','4356','4362','4368','4374','4380','4386','4392','4398','4404','4410','4416','4422','4428','4434','4440','4446','4452','4458','4464','4470','4476','4482','4488','4494','4500','4506','4512','4518','4524','4530','4536','4542','4548','4554','4560','4566','4572','4578','4584','4590','4596','4602','4608','4614','4620','4626','4632','4638','4644','4650','4656','4662','4668','4674','4680','4686','4692','4698','4704','4710','4716','4722','4728','4734','4740','4746','4752','4758','4764','4770','4776','4782','4788','4794','4800','4806','4812','4818','4824','4830','4836','4842','4848','4854','4860','4866','4872','4878','4884','4890','4896','4902','4908','4914','4920','4926','4932','4938','4944','4950','4956','4962','4968','4974','4980','4986','4992','4998','5004','5010','5016','5022','5028','5034','5040','5046','5052','5058','5064','5070','5076','5082','5088','5094','5100','5106','5112','5118','5124','5130','5136','5142','5148','5154','5160'],
             'format':'grib'      }

